# Fluid API

In [13]:
#%conda install -c conda-forge -y pyyaml
#%conda install -c conda-forge -y dpath
#%conda install -c conda-forge -y openai

In [14]:
import dpath
import json
import yaml

EXAMPLE_SCHEMA = """
paths:
  /sum:
    post:
      summary: Returns the sum of two numbers a and b.
      requestBody:
        required: true
        content:
          application/json:
            schema: 
              type: object
              properties:
                params:
                  type: object
                  properties: 
                    a: number
                    b: number
                    c: string
                    d: string
"""

EXAMPLE_TRANSFORMER = [
    {"from": ["params/a"], "to": ["a"], "type": "number"},
    {"from": ["params/b"], "to": ["b"], "type": "number", "split": " ", "format": "{0}"},
    {"from": ["params/one", "params/two"], "to": ["c"], "format": "{0} {1}", "type": "string"},
    {"from": ["params/one", "params/two"], "to": ["d"], "join": " ", "type": "string"},
    {"from": ["params/*"], "to": ["e"], "type": "list"},
]

TRANSFORMER_MAP = {}


def generate_transformer(input_example, target_schema):
    # import openai...
    # Add example input and example output before asking for transformation
    # Role: "Your task is to create a transformer from the provided YAML input example to match the provided OpenAPI schema"
    print("-" * 79)
    print(
f'''YAML input example: """{input_example}"""
Target OpenAPI schema: """{target_schema}"""
Transformer: """{EXAMPLE_TRANSFORMER}"""'''
    )
    print("-" * 79)
    return EXAMPLE_TRANSFORMER


def get_transformer(request, schema):
  if schema not in TRANSFORMER_MAP:
    TRANSFORMER_MAP[schema] = [generate_transformer(request, schema)]
  transformer_list = TRANSFORMER_MAP[schema]
  for transformer in transformer_list:
    print(f'request: {request}')
    misses = 0
    for transformation in transformer:
      print(transformation)
      for path in transformation["from"]:
        print(f'path: {path}')
        matches = dpath.search(request, path)
        if not matches:
          misses += 1
          print(f'misses: {misses}')
          break
        else:
          print(f'matches: {matches}')
    if misses == 0:
      return transformer
  if misses > 0:
    transformer = generate_transformer(request, schema)
    TRANSFORMER_MAP[schema] += [transformer]
  return transformer


def transform(request, schema):
    # TODO: On exception from here schedulle regeneration of transformer if failed is older than T seconds
    transformer = get_transformer(request, schema)
    print("-" * 79)
    print(transformer)
    response = {}
    for transformation in transformer:
      print(transformation)
      values = []
      value = None
      for path in transformation["from"]:
        print(f'path: {path}')
        values += dpath.values(request, path)
      print(f'values: {values}')
      if "split" in transformation:
        splitter = transformation["split"]
        values = [item for sublist in [str(x).split(splitter) for x in values] for item in sublist]
      if "format" in transformation:
        value = transformation["format"].format(*values)
        values = [value]
      elif "join" in transformation:
        value = transformation["join"].join(values)
        values = [value]
      typename = None
      if "type" in transformation:
        typename = transformation["type"]
      if len(values) == 1:
        value = values[0]
      else:
        value = values
      if typename == "object":
        if not isinstance(value, object):
          value = {}
          for item in values:
            value = value.update(item)
      elif typename == "list":
        if not isinstance(value, list):
          value = list(value)
      elif typename == "string":
        if not isinstance(value, str):
          value = str(value)
      elif typename == "number":
        if not isinstance(value, float):
          value = float(value)
      elif typename == "integer":
        if not isinstance(value, int):
          value = int(value)
      print(f'value: {value}')
      for destination in transformation["to"]:
        response[destination] = value
    return response


def api_gateway(request, schema):
    request_data = json.loads(request)
    request_yaml = yaml.dump(request_data, allow_unicode=True)
    transformed_data = transform(request_data, schema)
    response_yaml = yaml.dump(transformed_data, allow_unicode=True)
    print("-" * 79)
    print(request_yaml)
    print("=" * 79)
    print(response_yaml)


request = json.dumps(yaml.safe_load("""
params:
  a: 4.1
  b: "2 sausages"
  one: "amoeba"
  two: "mosquito"
"""))

api_gateway(request, EXAMPLE_SCHEMA)


-------------------------------------------------------------------------------
YAML input example: """{'params': {'a': 4.1, 'b': '2 sausages', 'one': 'amoeba', 'two': 'mosquito'}}"""
Target OpenAPI schema: """
paths:
  /sum:
    post:
      summary: Returns the sum of two numbers a and b.
      requestBody:
        required: true
        content:
          application/json:
            schema: 
              type: object
              properties:
                params:
                  type: object
                  properties: 
                    a: number
                    b: number
                    c: string
                    d: string
"""
Transformer: """[{'from': ['params/a'], 'to': ['a'], 'type': 'number'}, {'from': ['params/b'], 'to': ['b'], 'type': 'number', 'split': ' ', 'format': '{0}'}, {'from': ['params/one', 'params/two'], 'to': ['c'], 'format': '{0} {1}', 'type': 'string'}, {'from': ['params/one', 'params/two'], 'to': ['d'], 'join': ' ', 'type': 'string'}, {'fr